In [ ]:
import numpy as np
import scipy.io as sio
from scipy import signal
from scipy import stats
from nitime.algorithms import multi_taper_psd
from nitime.utils import dpss_windows
import statsmodels.api as sm
from statsmodels.genmod.families import Poisson
import statsmodels.formula.api as smf
from pandas import DataFrame as df
from pandas import concat
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import chi2, norm
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, xlim, ylim, show, imshow, xlabel, ylabel, bar, figure, title
from matplotlib import rcParams
from numpy import hanning, where, histogram, arange
rcParams['figure.figsize'] = (15,5)

Refaça a análise dos spikes da aula de hoje usando as ferramentas que aprendemos na aula I de spikes (ISI, Fator Fano, Autocorrelação) e analise o LFP. O resultado dessas análises ajuda a explicar o resultado da coerência entre campo e spike?

In [ ]:
#ISI

data = sio.loadmat('Aula15-spikes-LFP-1.mat')
Spikes = data['n'][0]

ISI = np.diff(Spikes)

plot(Spikes, np.ones_like(Spikes), '*', color = "navy")    
plt.xlim([-5, 5])              
plt.ylim([0, 3])              
xlabel('Time [s]')            
plt.legend({'Spikes'})   
show()


In [ ]:
#Fator Fano

FF = IncrementsLow50.var() / IncrementsLow50.mean()
print('FF =', FF50Low)

In [ ]:
#Autocorrelação

def autocorr(x, lags):
    xcorr = np.correlate(x - x.mean(), x - x.mean(), 'full')  
    xcorr = xcorr[xcorr.size//2:] / xcorr.max()               
    return xcorr[:lags+1]

Increments50, _ = np.histogram(Spikes, time_bins) 
autocorr(Increments50, 3)

time_bins = np.arange(0, 50, 0.001)                    
Increments50, _ = np.histogram(Spikes, time_bins) 
ACFLow = autocorr(Increments50, 100)                 

plot(ACFLow, '*', color = "red")        
N1 = len(Increments50)                    
sig = 2 / np.sqrt(N1)                       
plot([0, 100], [sig, sig], 'r:', '|', color = "navy")           
plot([0, 100], [-sig, -sig], 'r:', '|', color = "navy")
plt.xlim([0, 100])                         
plt.ylim([-.1, .1])                        
show()

Faça a análise abaixo para os dados Aula15-spikes-LFP2.mat e Aula15-spikes-LFP3.mat

In [ ]:
#Visualize os dados. Tem algum ritmo visível? É possível enxergar alguma associação entre spikes e o LFP?


data2 = sio.loadmat('Aula15-spikes-LFP-2.mat')
y = data['y']                               
n = data['n']                                
t = data['t'].reshape(-1)                    
plot(t,y[1,:])                               

plot(t,n[1,:])
xlabel('Time [s]')
plt.autoscale(tight=True)



data3 = sio.loadmat('Aula15-spikes-LFP-3.mat')

y = data3['y']                               
n = data3['n']                                
t = data3['t'].reshape(-1)                    
plot(t,y[1,:])                               

plot(t,n[1,:])
xlabel('Time [s]')
plt.autoscale(tight=True)

